<a href="https://colab.research.google.com/github/greenty5/ict2022_fluidDynamics/blob/main/Session24_particleTrace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fluid Dynamics with Python (ICT Summer School 2022)

Section 2.3 Flow visualization processing
=====
***

In this jupyterLab file, let us learn how to visualize fluid flow by using experimentally video-recording particle movies. 

Please don't run this jupyterLab file on the Google Colabratory system because this processing is too big to the system. 
So, please download this file on your computer when you want to do the visualization experiments

---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 # Open Source Computer Vision Library

In [ ]:
cap = cv2.VideoCapture( 'IMG_3057.MOV' ) # Input file
#cap = cv2.VideoCapture( 'input.mov' ) # Input file
isFirst = True  # sw for beginning frame 
while( cap.isOpened() ):
    ret, frame = cap.read()     # Loading frames from input movie
    if ret:
        if isFirst: # i==0: #isFirst:
            prvs = frame[ :, :, 2 ] # save as previous image (using green color)
            # color matrix for flow velocity
            rgb = np.zeros_like( frame ); rgb[...,0] = 0 # initializing (black)
            fourcc = cv2.VideoWriter_fourcc(*'MJPG') # setting for output movie codec
            # output movie setting: prvs(640,426,3) >> prvs.shape[::-1] (3,426,640)
            out = cv2.VideoWriter( 'result.avi', fourcc, 30, prvs.shape[ ::-1 ] )            
            # XY coordinate for felocity data (converting openCV data format to matplotlib data format)
            X, Y = np.meshgrid( np.arange( 0, frame.shape[1] ), 
                                           np.arange( frame.shape[0], 0, -1 ) )
        else:
            next = frame[ :, :, 2 ]
            flow = cv2.calcOpticalFlowFarneback(  # optical flow calculation
               prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            prvs = next  # saving calculated frames to prvs frames
            rgb[..., 2] = cv2.normalize( flow[...,1], None, 0, 255, cv2.NORM_MINMAX )
            rgb[..., 1] = 255 - rgb[..., 2]            
            # flow field (rhs ... openCV coordinate, lhs ... matplotlib coordinate)
            dx = flow[...,0]; dy = flow[...,1] 
            # drawing flow field using matplotlib
            fig = plt.figure( figsize = prvs.shape[::-1], dpi=1 )
            fig.patch.set_facecolor('black') 
            ax = fig.gca();  ax.axis('off')
            plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
            pitch = 15      # interval of flow field drawing
            plt.quiver( X[::pitch, ::pitch],    Y[ ::pitch, ::pitch ], 
                           dx[::pitch, ::pitch], -dy[::pitch, ::pitch],
                np.sqrt( dx[::pitch, ::pitch]**2 + dy[::pitch, ::pitch]**2 ),
                units="xy", scale=0.05, cmap="RdYlBu",
                linewidth=7 )
            fig.canvas.draw() # saving rendering results
            image = np.array( fig.canvas.renderer._renderer )
            plt.close(fig)
            # save result as a movie
            rgb = cv2.addWeighted( rgb, 0.5, image[...,0:3], 0.5, 1.0)
            out.write( rgb )
        isFirst = False
    else:
        cap.release(); out.release() # close input and output movies
        break

---
This file is based on a code developed by @hirax
(Book: Python Programing (ISBN 978-4-297-11637-8) 2020 by Hirabayashi Jun (Gihyou-sya))